In [1]:
import os
import textwrap
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from datasets import load_dataset
ds = load_dataset(
    "hwaseem04/Aya-testing",
    data_files={"xGQA_vqa": "data/xGQA_vqa-00000-of-00001.parquet"}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


xGQA_vqa-00000-of-00001.parquet:   0%|          | 0.00/340M [00:00<?, ?B/s]

Generating xGQA_vqa split: 0 examples [00:00, ? examples/s]

In [8]:
ds['xGQA_vqa'][0]


{'sample_id': 1,
 'image_id': 'n161313',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x280>,
 'question_en': 'Is it overcast?',
 'answer_en': 'No, it is clear.',
 'question_bn': 'এটা কি মেঘাচ্ছন্ন?',
 'answer_bn': 'না, এটা স্পষ্ট।',
 'question_de': 'Ist es bewölkt?',
 'answer_de': 'Nein, es ist klar.',
 'question_id': 'Apakah langit mendung ?',
 'answer_id': 'Tidak, itu jelas.',
 'question_ko': '날이 흐린가요?',
 'answer_ko': '아니, 분명해요.',
 'question_ru': 'Это пасмурная погода?',
 'answer_ru': 'Нет, это ясно.',
 'question_zh': '是阴天吗？',
 'answer_zh': '不，很清楚。'}

In [10]:
# Standard setup
from tqdm import tqdm
from PIL import Image
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

# Load Gemma-3 model and processor
model_id = "google/gemma-3-4b-it"
model = Gemma3ForConditionalGeneration.from_pretrained(model_id, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(model_id)

# Languages to iterate over
languages = ["en", "bn", "de", "ko", "ru", "zh"]

# Directory to save temp images (needed for this model)
os.makedirs("temp_images_vqa", exist_ok=True)

dataset = ds['xGQA_vqa']

for sample in tqdm(dataset, desc="Iterating samples"):
    try:
        image = sample["image"]
        sample_id = sample["sample_id"]

        # Save the image locally if not already present
        image_path = f"temp_images_vqa/{sample_id}.jpg"
        if not os.path.exists(image_path):
            image.save(image_path)

        print(f"\n========== Sample ID: {sample_id} ==========")

        for lang in languages:
            question_col = f"question_{lang}"
            answer_col = f"answer_{lang}"

            # Safety check if expected keys exist
            if question_col not in sample or answer_col not in sample:
                print(f"[{lang}] Missing data.")
                continue

            question = sample[question_col]
            gt_answer = sample[answer_col]

            # Create chat-format input as expected by Gemma
            messages = [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": "You are a helpful assistant."}]
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image_path},
                        {"type": "text", "text": question}
                    ]
                }
            ]

            # Format input for the model
            inputs = processor.apply_chat_template(
                messages,
                add_generation_prompt=True,
                tokenize=True,
                return_dict=True,
                return_tensors="pt"
            ).to(model.device, dtype=torch.bfloat16)

            input_len = inputs["input_ids"].shape[-1]

            # Generate prediction using Gemma
            with torch.inference_mode():
                generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
                generation = generation[0][input_len:]

            # Decode output
            pred_answer = processor.decode(generation, skip_special_tokens=True)

            # Display result
            print(f"\n[{lang.upper()}]")
            print(f"Question: {question}")
            print(f"GT: {gt_answer}")
            print(f"Pred: {textwrap.fill(pred_answer, width=80)}")

        print("=" * 100)

    except Exception as e:
        print(f"Error processing sample {sample['sample_id']}: {e}")

    break

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Iterating samples:   0%|          | 0/9666 [00:00<?, ?it/s]


========== Sample ID: 1 ==========

[EN]
Question: Is it overcast?
GT: No, it is clear.
Pred: No, it is not overcast. The sky in the image is a clear, bright blue.

[BN]
Question: এটা কি মেঘাচ্ছন্ন?
GT: না, এটা স্পষ্ট।
Pred: ছবিটিতে আকাশ উজ্জ্বল এবং মেঘ নেই। এটি একটি পরিষ্কার, নীল দিন।

[DE]
Question: Ist es bewölkt?
GT: Nein, es ist klar.
Pred: Nein, es ist nicht bewölkt. Es ist ein blauer Himmel.

[KO]
Question: 날이 흐린가요?
GT: 아니, 분명해요.
Pred: 아니요, 날씨가 맑습니다. 하늘은 파랗습니다.

[RU]
Question: Это пасмурная погода?
GT: Нет, это ясно.
Pred: Нет, это не пасмурная погода. На фотографии ярко-голубое небо.


Iterating samples:   0%|          | 0/9666 [00:09<?, ?it/s]


[ZH]
Question: 是阴天吗？
GT: 不，很清楚。
Pred: 根据这张照片，天空是蓝色的，所以不是阴天。
